In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model


In [39]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Hands-on Generative AI Course', 'language': 'en'}, page_content="Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 16th September 2025Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · 3 classes/week · 2 hrs/class + Post-class Doubt SupportClasses on Tue, Wed, Thurs - 9PM ISTAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Gene

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [5]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 16th September 2025Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · 3 classes/week · 2 hrs/class + Post-class Doubt SupportClasses on Tue, Wed, Thurs - 9PM ISTAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (VAEs) Probabilistic Data Generation Using VAEs Four Mini Project

In [6]:
print(len(splits))

11


In [10]:
import chromadb
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize a Hugging Face embedding model.
# A popular choice is 'sentence-transformers/all-MiniLM-L6-v2'
# This model is small, fast, and effective.
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_function = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Create the vector store with your documents and the new embedding function
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

In [11]:
print(vectorstore._collection.count())

11


In [12]:
print("\nCollection 1 - ", vectorstore._collection.get(ids=['28651d9a-ab51-41f8-ab83-e68285623c4e'], include=["embeddings", "documents"]))
print("\nCollection 2 - ", vectorstore._collection.get(ids=['054dee19-19ed-4574-bc51-511060fd707a'], include=["embeddings", "documents"]))
print("\nCollection 3 - ", vectorstore._collection.get(ids=['2fd71cb4-835a-43c5-b920-b7e1be51c450'], include=["embeddings", "documents"]))


Collection 1 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 2 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 3 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}


In [13]:
retriever = vectorstore.as_retriever()

In [14]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [25]:
from langchain_groq import ChatGroq
import os

# Ensure your Groq API key is set as an environment variable
os.environ["GROQ_API_KEY"] = "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"

# Initialize the Groq language model
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",  # Choose a model available on Groq
    temperature=0  # Set the desired temperature
)

In [27]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [28]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [29]:
rag_chain = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [30]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'The course recordings will be available for lifetime access. You can access them at any time, even after the course has ended.'

In [31]:
rag_chain.invoke("Are the testimonials for the course available? Name the studenst who have shared testimonials")

"Yes, the testimonials for the course are available. The students who have shared testimonials are Keerti's students, including Manika Kaushik, Sahitya Raj A, and Keerti's students who are not named."

In [32]:
rag_chain.invoke("Are the certificates for the course provided?")

'Yes, the certificates for the course are provided. This information can be found in the FAQ section of the context.'

In [33]:
rag_chain.invoke("What all projects are covered in the course?")

"Unfortunately, the context doesn't explicitly mention the projects covered in the course. However, it does mention that the course covers Generative AI and Machine Learning, and that students work on multiple projects."

In [34]:
from langchain_core.runnables import RunnableLambda

In [35]:
def print_prompt(prompt_text):
  print("Prompt - ", prompt_text)
  return prompt_text

In [36]:
rag_chain_with_print = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | RunnableLambda(print_prompt)
             | llm
             | StrOutputParser())

In [37]:
rag_chain_with_print.invoke("What all projects are covered in the course?")

Prompt -  messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What all projects are covered in the course? \nContext: knowledge of Machine Learning. Keerti’s passion for teaching made complex topics easy to grasp. I highly recommend this course to anyone interested in AI and ML!Read moreManika KaushikSenior Software EngineerOptum-United HealthGroupKeerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.Frequently asked questionsIs this a Live or Recorded Course?When will the next Live batch be launched?What if I am interested in learning Live only?What are the prerequisites for the course?Is Machine Learning pre-requisite for the course?How many projects will we work on? Can I add these to resume?I

"Unfortunately, the context doesn't explicitly mention the projects covered in the course. However, it does mention that the course covers Generative AI and Machine Learning, and that students work on multiple projects."